In [ ]:
import numpy as np
import pandas as pd
import random
import math
import time

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score

In [ ]:
class DecisionBinaryTree():

    def __init__(self, label):
        """
        Build a binary tree class adapted to the Decision Tree architecture using the Node class implemented above.
        """
        self.label = label
        self.split_feature = None
        self.split_value = None
        self.left_child = None
        self.right_child = None
        self.depth = 0

    def get_node(self, path):
        """
        Runs through the tree given input path and returns corresponding node.
        """
        node = self
        for dir_ in path:
            if dir_ == "left":
                node = node.left_child
            elif dir_ == "right":
                node = node.right_child
        return node

    def add_split(self, path, split_feature, split_value, left_label, right_label):
        """
        Adds a split at the given path and using the given information.
        """
        node = self.get_node(path)
        node.split_feature = split_feature
        node.split_value = split_value
        node.left_child = DecisionBinaryTree(left_label)
        node.right_child = DecisionBinaryTree(right_label)
        # Increment tree depth if path is as long as depth
        if len(path) >= self.depth:
            self.depth += 1

In [ ]:
def gini(y,split_index):
  left_split, right_split = y[:split_index], y[split_index:]

  # Compute left and right gini index
  left_gini = 1 - np.sum(np.square(np.unique(left_split, return_counts=True)[1] / left_split.shape[0]))
  right_gini = 1 - np.sum(np.square(np.unique(right_split, return_counts=True)[1] / right_split.shape[0]))
  
  # Return weighted means of gini index
  return (left_split.shape[0] * left_gini / X.shape[0]) + (right_split.shape[0] * right_gini / X.shape[0])

In [ ]:
def crossentropy(y,split_index):
  left_split, right_split = y[:split_index], y[split_index:]

  # Compute left and right crossentropy
  left_probs, right_probs = np.unique(left_split, return_counts=True)[1] / left_split.shape[0], np.unique(right_split, return_counts=True)[1] / right_split.shape[0]
  left_crossentropy = - np.sum(left_probs * np.log2(left_probs))
  right_crossentropy = - np.sum(right_probs * np.log2(right_probs))

  return (left_split.shape[0] * left_crossentropy / X.shape[0]) + (right_split.shape[0] * right_crossentropy / X.shape[0])

In [ ]:
def get_best_split(indices,X,y,criterion,min_samples_split):
    """
    Returns best split value along with indexes of the left and right leaf it creates.
    Args:
        indices (np.array): sample indices
        X (np.array): feature vector
        y (np.array): label vector
        feature (int): index of split feature
        criterion (String): either "gini" or "crossentropy", split metric
        min_samples_split (int): number of minimum samples that must be contained in each leaf of the split
    """
    # Sort indices 
    sort_indices = np.argsort(X)
    indices_sorted, X_sorted, y_sorted = indices[sort_indices], X[sort_indices], y[sort_indices]

    # Find best split
    best_split_index, best_criterion_value = None, None
    for i in range(min_samples_split-1,X_sorted.shape[0]-min_samples_split): # Only run through possible splits with regards to min_samples_split
        if criterion == "gini":
            criterion_value = gini(y_sorted, i)
        elif criterion == "crossentropy":
            criterion_value = crossentropy(y_sorted, i)
        else:
            raise ValueError("""Wrong criterion! Must be in ["gini","crossentropy"]""")

        # Update best split
        if best_criterion_value is None:
            best_criterion_value, best_split_index = criterion_value, i
        else:
            if best_criterion_value > criterion_value:
                best_criterion_value, best_split_index = criterion_value, i
    
    # Return best criterion value, split value and indexes
    return best_criterion_value, X_sorted[best_split_index], indices_sorted[:best_split_index+1], indices_sorted[best_split_index+1:]

In [ ]:
class DecisionTree_Classification():
    """
    Decision Tree Model. Will benefit from the same structure as the sklearn implementation.
    """

    def __init__(self, criterion="gini", max_depth=None, max_features=None, min_samples_split=2, random_state=None):
        """
        Attributes:
            criterion (string): split criterion, either "gini" or "crossentropy"
            max_depth (int): maximum tree depth
            max_features (int): number of features to consider when looking for a best split
            min_sample_split (int): minimum number of instances in each set in order to split
            random_state (int): random seed
            depth (int): tree depth
            splits (dict): dictionary containing split conditions, each leaf is described by the path taken to the leaf (list of "left" and "right")
            subsets (dict): maps each leaf to the subset of samples it describes
        """
        # Set input attribute values
        self.criterion = criterion
        self.max_depth = max_depth
        self.max_features = max_features    
        self.min_samples_split = 2
        self.random_state = random_state

        # Set initial tree state attributes
        self.depth = 0
        self.splits = DecisionBinaryTree(None)
        self.subsets = {}

    def fit(self, X, y, verbose=True):
        """
        Fit model with input data.
        Args:
            X (np.array): feature array of dimension (nb_samples,nb_features)
            y (np.array): label array of dimension (nb_samples)
        """
        start = time.time()
        # Set initial state as one complete set of all data
        self.subsets["[]"] = np.arange(X.shape[0])
        # Set random seed, if any is given
        if not self.random_state is None:
            np.random.seed(self.random_state)

        # Split while a break condition has not been met
        break_condition = False
        while not break_condition:
            leafs = list(self.subsets.items()) # Get leafs and their corresponding train samples
            # Set list of features on which split is applied
            if self.max_features is None:
                split_features = np.arange(X.shape[1]) # If no feature constraint is given, all features must be considered for split
            else:
                nb_features = min(self.max_features,X.shape[1]) # Take the min between the total number of features and the max_features value
                split_features = np.random.choice(np.arange(X.shape[1]), nb_features) # Consider a random sample of nb_features features on which to apply the split
            best_split_criterion  = None # Contains best split attributes

            # Run through each leaf and find best split
            for leaf in leafs:
                leaf_path, leaf_set = leaf
                # Check if current leaf can be extended
                if (not self.max_depth is None) and (len(eval(leaf_path)) == self.max_depth): # If maximum tree depth is met
                    continue

                # Check if leaf has enough elements to be split
                if len(leaf_set) < 2 * self.min_samples_split:
                    continue

                X_subset, y_subset = X[leaf_set,:], y[leaf_set]
                
                best_leaf_criterion = None # Instantiate best leaf split
                for feature in split_features:
                    X_subset_feat = X_subset[:,feature]
                    best_feat_criterion, feat_split_value, left_indices_feat, right_indices_feat = get_best_split(leaf_set, X_subset_feat, y_subset, self.criterion, self.min_samples_split)

                    # Check if it is best split in leaf
                    if (best_leaf_criterion) is None or (best_leaf_criterion > best_feat_criterion):
                        leaf_split_feature, best_leaf_criterion, leaf_split_value, left_indices_leaf, right_indices_leaf = feature, best_feat_criterion, feat_split_value, left_indices_feat, right_indices_feat

                # Check if leaf has the best split
                if (best_split_criterion is None) or (best_split_criterion > best_leaf_criterion):
                    best_leaf, split_feature, best_split_criterion, split_value, left_indices, right_indices = leaf_path, leaf_split_feature, best_leaf_criterion, leaf_split_value, left_indices_leaf, right_indices_leaf

            # Best split has been found, check if valid. If so, create new split, else break
            if best_split_criterion is None: # If no split criterion has been found, break (e.g. all nodes have small amounts of data)
                break_condition = True
            else:
                if verbose:
                    print("Split of leaf:",best_leaf)
                    print("   | On feature:",split_feature)
                    print("   | At value:", split_value)
                
                # Update splits and subsets attributes
                best_leaf_path = eval(best_leaf) # Get path to best leaf as list

                self.subsets.pop(best_leaf) # Remove old leaf
                left_key, right_key = str(best_leaf_path + ["left"]), str(best_leaf_path + ["right"])
                self.subsets[left_key] = left_indices
                self.subsets[right_key] = right_indices

                # Compute label of each split
                left_labels, right_labels = y[left_indices], y[right_indices]
                left_vals, left_occs = np.unique(left_labels, return_counts=True)
                right_vals, right_occs = np.unique(right_labels, return_counts=True)
                left_label, right_label = left_vals[np.argmax(left_occs)], right_vals[np.argmax(right_occs)]

                self.splits.add_split(best_leaf_path, split_feature, split_value, left_label, right_label)

        stop = time.time()
        print("Training time:",round(stop-start,2),"s")


    def unique_pred(self, X_row):
        """
        Make a prediction on input row. Run through the binary split tree until a leaf is encountered.
        """
        # Start from top of the split tree and run down to a leaf according to the split conditions
        node = self.splits
        while not node.left_child is None: # While node is not terminal
            split_feature, split_value = node.split_feature, node.split_value
            if X_row[split_feature] <= split_value: # Go to the left child
                node = node.left_child
            else:
                node = node.right_child
        return node.label

    def predict(self, X):
        return np.apply_along_axis(self.unique_pred, 1, X)

    def accuracy_score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y,y_pred)

### Compare Results with SK-learn

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

X,y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

We first train an sklearn Decision Tree Classifier with a maximum depth of 4:

In [ ]:
dt_sklearn = DecisionTreeClassifier(max_depth=4)

# Train
start = time.time()
dt_sklearn.fit(X_train,y_train)
stop = time.time()
print("Training Time:",stop-start,"s")

Training Time: 0.012940168380737305 s


In [ ]:
# Make test prediction
print("Test Accuracy:",dt_sklearn.score(X_test,y_test))

Test Accuracy: 0.9385964912280702


We now train our own model on the same dataset, with the same parameters:

In [ ]:
dt = DecisionTree_Classification(max_depth = 4)
dt.fit(X_train,y_train)

Split of leaf: []
   | On feature: 7
   | At value: 0.05182
Split of leaf: ['left']
   | On feature: 20
   | At value: 16.82
Split of leaf: ['left', 'left']
   | On feature: 10
   | At value: 0.645
Split of leaf: ['left', 'left', 'left']
   | On feature: 13
   | At value: 41.0
Split of leaf: ['right']
   | On feature: 27
   | At value: 0.1466
Split of leaf: ['right', 'right']
   | On feature: 16
   | At value: 0.1278
Split of leaf: ['right', 'right', 'left']
   | On feature: 0
   | At value: 11.42
Split of leaf: ['left', 'right']
   | On feature: 1
   | At value: 16.68
Split of leaf: ['left', 'right', 'left']
   | On feature: 11
   | At value: 0.3628
Split of leaf: ['left', 'right', 'right']
   | On feature: 11
   | At value: 1.439
Split of leaf: ['right', 'left']
   | On feature: 22
   | At value: 116.2
Split of leaf: ['right', 'left', 'left']
   | On feature: 1
   | At value: 21.35
Split of leaf: ['right', 'left', 'right']
   | On feature: 0
   | At value: 16.6
Training time: 9.08 s


In [ ]:
print("Test Accuracy:",dt.accuracy_score(X_test,y_test))

Test Accuracy: 0.9473684210526315
